In [579]:
using Pkg
pkg"add Oceananigans, NCDatasets, Plots, Printf, Polynomials"

   Resolving package versions...
  No Changes to `C:\Users\writi\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\writi\.julia\environments\v1.7\Manifest.toml`


In [580]:
using Oceananigans
using Oceananigans.Models: ShallowWaterModel

In [581]:
Lx, Ly, Lz = 2π, 20, 1
Nx, Ny = 128, 128

grid = RectilinearGrid(size = (Nx, Ny),
                       x = (0, Lx), y = (-Ly/2, Ly/2),
                       topology = (Periodic, Bounded, Flat))

128×128×1 RectilinearGrid{Float64, Periodic, Bounded, Flat} on CPU with 3×3×0 halo
├── Periodic x ∈ [-7.51279e-18, 6.28319) regularly spaced with Δx=0.0490874
├── Bounded  y ∈ [-10.0, 10.0]           regularly spaced with Δy=0.15625
└── Flat z

In [582]:
u_forcing_func(x, y, t, A, h) = (1/h)*(∂x(A)*∂y(∂y(A)/h) - ∂y(A)*∂x(∂y(A)/h))

v_forcing_func(x, y, t, A, h) = (1/h)*(∂x(A)*∂y(∂x(A)/h) - ∂y(A)*∂x(∂x(A)/h))

v_forcing_func (generic function with 1 method)

In [583]:
u_forcing = Forcing(u_forcing_func, field_dependencies = (:A, :h))

v_forcing = Forcing(v_forcing_func, field_dependencies = (:A, :h))

ContinuousForcing{Nothing}
├── func: v_forcing_func (generic function with 1 method)
├── parameters: nothing
└── field dependencies: (:A, :h)

In [584]:
const U = 1.0         # Maximum jet velocity

f = 1         # Coriolis parameter
g = 9.8         # Gravitational acceleration
Δη = f * U / g  # Maximum free-surface deformation as dictated by geostrophy

0.1020408163265306

In [585]:
model = ShallowWaterModel(
                          timestepper = :RungeKutta3,
                          advection = WENO5(),
                          grid = grid,
                          gravitational_acceleration = g,
                          coriolis = FPlane(f=f),
                          tracers = (:A),
                          forcing = (u = u_forcing, v = v_forcing))

┌ Warning: defaulting to uniform WENO scheme with Float64 precision, use WENO5(grid = grid) if this was not intended
└ @ Oceananigans.Advection C:\Users\writi\.julia\packages\Oceananigans\SbcJ5\src\Advection\weno_fifth_order.jl:154


ShallowWaterModel{typename(CPU), Float64}(time = 0 seconds, iteration = 0) 
├── grid: 128×128×1 RectilinearGrid{Float64, Periodic, Bounded, Flat} on CPU with 3×3×0 halo
├── tracers: (:A,)
└── coriolis: FPlane{Float64}

In [586]:
h̄(x, y, z) = Lz - Δη * tanh(y)
ū(x, y, z) = U * sech(y)^2

ū (generic function with 1 method)

In [587]:
ω̄(x, y, z) = 2 * U * sech(y)^2 * tanh(y)

ω̄ (generic function with 1 method)

In [588]:
small_amplitude = 1e-4

 uⁱ(x, y, z) = ū(x, y, z) + small_amplitude * exp(-y^2) * randn()
uhⁱ(x, y, z) = uⁱ(x, y, z) * h̄(x, y, z)

uhⁱ (generic function with 1 method)

In [589]:
ū̄h(x, y, z) = ū(x, y, z) * h̄(x, y, z)

A_i(x, y, z) = -y + randn()

set!(model, uh = ū̄h, h = h̄, A = A_i)

In [590]:
uh, vh, h = model.solution

# Build velocities
u = uh / h
v = vh / h

# Build and compute mean vorticity discretely
ω = Field(∂x(v) - ∂y(u))
compute!(ω)

# Copy mean vorticity to a new field
ωⁱ = Field{Face, Face, Nothing}(model.grid)
ωⁱ .= ω

# Use this new field to compute the perturbation vorticity
ω′ = Field(ω - ωⁱ)

128×129×1 Field{Face, Face, Center} on RectilinearGrid on CPU
├── grid: 128×128×1 RectilinearGrid{Float64, Periodic, Bounded, Flat} on CPU with 3×3×0 halo
├── boundary conditions: FieldBoundaryConditions
│   └── west: Periodic, east: Periodic, south: Nothing, north: Nothing, bottom: Nothing, top: Nothing, immersed: ZeroFlux
├── operand: BinaryOperation at (Face, Face, Center)
├── status: time=0.0
└── data: 134×135×1 OffsetArray(::Array{Float64, 3}, -2:131, -2:132, 1:1) with eltype Float64 with indices -2:131×-2:132×1:1
    └── max=0.0, min=0.0, mean=0.0

In [591]:
set!(model, uh = uhⁱ)

In [592]:
simulation = Simulation(model, Δt = 1e-2, stop_time = 300)

Simulation of ShallowWaterModel{RectilinearGrid{Float64, Periodic, Bounded, Flat, Float64, Float64, Float64, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}, CPU}, CPU, Float64, WENO5{Float64, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, true}, FPlane{Float64}, NamedTuple{(:uh, :vh, :h, :A), NTuple{4, typeof(Oceananigans.Forcings.zeroforcing)}}, Nothing, Nothing, NamedTuple{(:uh, :vh, :h), Tuple{Field{Face, Center, Center, Nothing, RectilinearGrid{Float64, Periodic, Bounded, Flat, Float64, Float64, Float64, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}, OffsetArrays.OffsetVector{Float64, StepRangeLen{Fl

In [593]:
using LinearAlgebra: norm

perturbation_norm(args...) = norm(v)

perturbation_norm (generic function with 1 method)

In [594]:
simulation.output_writers[:fields] = NetCDFOutputWriter(model, (; ω, ω′),
                                                        filename = joinpath(@__DIR__, "shallow_water_Bickley_jet_fields.nc"),
                                                        schedule = TimeInterval(1),
                                                        overwrite_existing = true)

┌ Warning: Overwriting existing C:\windows\system32\shallow_water_Bickley_jet_fields.nc.
└ @ Oceananigans.OutputWriters C:\Users\writi\.julia\packages\Oceananigans\SbcJ5\src\OutputWriters\netcdf_output_writer.jl:313


NetCDFOutputWriter scheduled on TimeInterval(1 second):
├── filepath: C:\windows\system32\shallow_water_Bickley_jet_fields.nc
├── dimensions: zC(1), zF(1), xC(128), yF(129), xF(128), yC(128), time(0)
├── 2 outputs: (ω, ω′)
└── array type: Array{Float32}

In [595]:
simulation.output_writers[:growth] = NetCDFOutputWriter(model, (; perturbation_norm),
                                                        filename = joinpath(@__DIR__, "shallow_water_Bickley_jet_perturbation_norm.nc"),
                                                        schedule = IterationInterval(1),
                                                        dimensions = (; perturbation_norm = ()),
                                                        overwrite_existing = true)

┌ Warning: Overwriting existing C:\windows\system32\shallow_water_Bickley_jet_perturbation_norm.nc.
└ @ Oceananigans.OutputWriters C:\Users\writi\.julia\packages\Oceananigans\SbcJ5\src\OutputWriters\netcdf_output_writer.jl:313


NetCDFOutputWriter scheduled on IterationInterval(1):
├── filepath: C:\windows\system32\shallow_water_Bickley_jet_perturbation_norm.nc
├── dimensions: zC(1), zF(1), xC(128), yF(129), xF(128), yC(128), time(0)
├── 1 outputs: perturbation_norm
└── array type: Array{Float32}

In [596]:
run!(simulation)

┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations C:\Users\writi\.julia\packages\Oceananigans\SbcJ5\src\Simulations\run.jl:167
┌ Info:     ... simulation initialization complete (27.364 ms)
└ @ Oceananigans.Simulations C:\Users\writi\.julia\packages\Oceananigans\SbcJ5\src\Simulations\run.jl:202
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations C:\Users\writi\.julia\packages\Oceananigans\SbcJ5\src\Simulations\run.jl:112
┌ Info:     ... initial time step complete (23.479 ms).
└ @ Oceananigans.Simulations C:\Users\writi\.julia\packages\Oceananigans\SbcJ5\src\Simulations\run.jl:119
┌ Info: Simulation is stopping. Model time 5.833 minutes has hit or exceeded simulation stop time 5.833 minutes.
└ @ Oceananigans.Simulations C:\Users\writi\.julia\packages\Oceananigans\SbcJ5\src\Simulations\simulation.jl:167


In [597]:
using NCDatasets, Plots, Printf

In [598]:
x, y = xnodes(ω), ynodes(ω)

([-7.512787384681048e-18, 0.04908738521234051, 0.09817477042468102, 0.14726215563702152, 0.19634954084936204, 0.24543692606170256, 0.2945243112740431, 0.3436116964863836, 0.39269908169872414, 0.4417864669110646  …  5.792311455056181, 5.841398840268521, 5.890486225480862, 5.939573610693202, 5.988660995905543, 6.037748381117884, 6.086835766330224, 6.135923151542564, 6.185010536754905, 6.234097921967245], [-10.0, -9.84375, -9.6875, -9.53125, -9.375, -9.21875, -9.0625, -8.90625, -8.75, -8.59375  …  8.59375, 8.75, 8.90625, 9.0625, 9.21875, 9.375, 9.53125, 9.6875, 9.84375, 10.0])

In [599]:
kwargs = (
         xlabel = "x",
         ylabel = "y",
         aspect = 1,
           fill = true,
         levels = 20,
      linewidth = 0,
          color = :balance,
       colorbar = true,
           ylim = (-Ly/2, Ly/2),
           xlim = (0, Lx)
)

(xlabel = "x", ylabel = "y", aspect = 1, fill = true, levels = 20, linewidth = 0, color = :balance, colorbar = true, ylim = (-10.0, 10.0), xlim = (0, 6.283185307179586))

In [600]:
ds = NCDataset(simulation.output_writers[:fields].filepath, "r")

anim = @animate for (iter, t) in enumerate(ds["time"])
    ω = ds["ω"][:, :, 1, iter]
    ω′ = ds["ω′"][:, :, 1, iter]

    ω′_max = maximum(abs, ω′)

    plot_ω = contour(x, y, ω',
                     clim = (-1, 1),
                     title = @sprintf("Total vorticity, ω, at t = %.1f", t); kwargs...)

    plot_ω′ = contour(x, y, ω′',
                      clim = (-ω′_max, ω′_max),
                      title = @sprintf("Perturbation vorticity, ω - ω̄, at t = %.1f", t); kwargs...)

    plot(plot_ω, plot_ω′, layout = (1, 2), size = (800, 440))
end

close(ds)

mp4(anim, "shallow_water_Bickley_jet.mp4", fps=15)

┌ Info: Saved animation to 
│   fn = C:\windows\system32\shallow_water_Bickley_jet.mp4
└ @ Plots C:\Users\writi\.julia\packages\Plots\nzdhU\src\animation.jl:114


Plots.AnimatedGif("C:\\windows\\system32\\shallow_water_Bickley_jet.mp4")